# Transform

In [1]:
import pathlib
import sys
import pickle

from typing import List

import numpy as np
import pandas as pd

# Load the "autoreload" extension
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1
# add the 'src' directory to path to import modules
src_dir = pathlib.Path().cwd().resolve().parent / 'src'
#src_dir = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(str(src_dir))
# import my class code from the source
# %aimport src-dir.filename
from data.data_process import transform_ts_to_supervised

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
import keras

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import roc_auc_score

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.set_printoptions(suppress=True)

PROJECT_DIR = pathlib.Path.cwd().resolve().parent
CONSOL_DATA_DIR = PROJECT_DIR / 'data' / '07-consolidated'

Using TensorFlow backend.


In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
fp = CONSOL_DATA_DIR / 'epl.csv'
df = pd.read_csv(fp, index_col=None)

In [4]:
df.head()

,nation,league,season,date,h,a,h_ftGoals,a_ftGoals,h_corners,h_fouls,h_redCards,h_shots,h_shotsOnTarget,h_htGoals,h_yellowCards,a_corners,a_fouls,a_redCards,a_shots,a_shotsOnTarget,a_htGoals,a_yellowCards,h_oddsWinMean,h_oddsDrawMean,a_oddsWinMean,oddsHwinBet,oddsDrawBet,oddsAwinBet,oddsHwinPinn,oddsDrawPinn,oddsAwinPinn,oddsHwinMax,oddsDrawMax,oddsAwinMax,result,h_result,a_result,h_goalsFor,h_goalsAgainst,a_goalsFor,a_goalsAgainst,h_impliedWinProbMean,h_impliedDrawProbMean,a_impliedDrawProbMean,a_impliedWinProbMean,a_oddsDrawMean,h_h_ftGoals-0,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,h_h_ftGoals-4,h_h_ftGoals-5,h_h_ftGoals-6,h_h_ftGoals-7,h_h_ftGoals-8,h_h_ftGoals-9,h_h_ftGoals-10,h_h_ftGoals-11,h_h_ftGoals-12,h_h_ftGoals-13,h_h_ftGoals-14,h_h_ftGoals-15,h_h_ftGoals-16,h_h_ftGoals-17,h_h_ftGoals-18,a_a_ftGoals-0,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_a_ftGoals-4,a_a_ftGoals-5,a_a_ftGoals-6,a_a_ftGoals-7,a_a_ftGoals-8,a_a_ftGoals-9,a_a_ftGoals-10,a_a_ftGoals-11,a_a_ftGoals-12,a_a_ftGoals-13,a_a_ftGoals-14,a_a_ftGoals-15,a_a_ftGoals-16,a_a_ftGoals-17,a_a_ftGoals-18,a_h_ftGoals-1,a_h_ftGoals-10,a_h_ftGoals-11,a_h_ftGoals-12,a_h_ftGoals-13,a_h_ftGoals-14,a_h_ftGoals-15,a_h_ftGoals-16,a_h_ftGoals-17,a_h_ftGoals-18,a_h_ftGoals-19,a_h_ftGoals-2,a_h_ftGoals-3,a_h_ftGoals-4,a_h_ftGoals-5,a_h_ftGoals-6,a_h_ftGoals-7,a_h_ftGoals-8,a_h_ftGoals-9,h_a_ftGoals-1,h_a_ftGoals-10,h_a_ftGoals-11,h_a_ftGoals-12,h_a_ftGoals-13,h_a_ftGoals-14,h_a_ftGoals-15,h_a_ftGoals-16,h_a_ftGoals-17,h_a_ftGoals-18,h_a_ftGoals-19,h_a_ftGoals-2,h_a_ftGoals-3,h_a_ftGoals-4,h_a_ftGoals-5,h_a_ftGoals-6,h_a_ftGoals-7,h_a_ftGoals-8,h_a_ftGoals-9,h_h_shots-0,h_h_shots-1,h_h_shots-2,h_h_shots-3,h_h_shots-4,h_h_shots-5,h_h_shots-6,h_h_shots-7,h_h_shots-8,h_h_shots-9,h_h_shots-10,h_h_shots-11,h_h_shots-12,h_h_shots-13,h_h_shots-14,h_h_shots-15,h_h_shots-16,h_h_shots-17,h_h_shots-18,a_a_shots-0,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_a_shots-4,a_a_shots-5,a_a_shots-6,a_a_shots-7,a_a_shots-8,a_a_shots-9,a_a_shots-10,a_a_shots-11,a_a_shots-12,a_a_shots-13,a_a_shots-14,a_a_shots-15,a_a_shots-16,a_a_shots-17,a_a_shots-18,a_h_shots-1,a_h_shots-10,a_h_shots-11,a_h_shots-12,a_h_shots-13,a_h_shots-14,a_h_shots-15,a_h_shots-16,a_h_shots-17,a_h_shots-18,a_h_shots-19,a_h_shots-2,a_h_shots-3,a_h_shots-4,a_h_shots-5,a_h_shots-6,a_h_shots-7,a_h_shots-8,a_h_shots-9,h_a_shots-1,h_a_shots-10,h_a_shots-11,h_a_shots-12,h_a_shots-13,h_a_shots-14,h_a_shots-15,h_a_shots-16,h_a_shots-17,h_a_shots-18,h_a_shots-19,h_a_shots-2,h_a_shots-3,h_a_shots-4,h_a_shots-5,h_a_shots-6,h_a_shots-7,h_a_shots-8,h_a_shots-9,h_h_shotsOnTarget-0,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,h_h_shotsOnTarget-4,h_h_shotsOnTarget-5,h_h_shotsOnTarget-6,h_h_shotsOnTarget-7,h_h_shotsOnTarget-8,h_h_shotsOnTarget-9,h_h_shotsOnTarget-10,h_h_shotsOnTarget-11,h_h_shotsOnTarget-12,h_h_shotsOnTarget-13,h_h_shotsOnTarget-14,h_h_shotsOnTarget-15,h_h_shotsOnTarget-16,h_h_shotsOnTarget-17,h_h_shotsOnTarget-18,a_a_shotsOnTarget-0,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_a_shotsOnTarget-4,a_a_shotsOnTarget-5,a_a_shotsOnTarget-6,a_a_shotsOnTarget-7,a_a_shotsOnTarget-8,a_a_shotsOnTarget-9,a_a_shotsOnTarget-10,a_a_shotsOnTarget-11,a_a_shotsOnTarget-12,a_a_shotsOnTarget-13,a_a_shotsOnTarget-14,a_a_shotsOnTarget-15,a_a_shotsOnTarget-16,a_a_shotsOnTarget-17,a_a_shotsOnTarget-18,a_h_shotsOnTarget-1,a_h_shotsOnTarget-10,a_h_shotsOnTarget-11,a_h_shotsOnTarget-12,a_h_shotsOnTarget-13,a_h_shotsOnTarget-14,a_h_shotsOnTarget-15,a_h_shotsOnTarget-16,a_h_shotsOnTarget-17,a_h_shotsOnTarget-18,a_h_shotsOnTarget-19,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,a_h_shotsOnTarget-4,a_h_shotsOnTarget-5,a_h_shotsOnTarget-6,a_h_shotsOnTarget-7,a_h_shotsOnTarget-8,a_h_shotsOnTarget-9,h_a_shotsOnTarget-1,h_a_shotsOnTarget-10,h_a_shotsOnTarget-11,h_a_shotsOnTarget-12,h_a_shotsOnTarget-13,h_a_shotsOnTarget-14,h_a_shotsOnTarget-15,h_a_shotsOnTarget-16,h_a_shotsOnTarget-17,h_a_shotsOnTarget-18,h_a_shotsOnTarget-19,h_a_shotsOnTa

In [5]:
import re

def get_n_past_games(df_orig, past_games=2, dropna_rows=True):
    """
    Restrict historical records to the last n games
    If n = 2, then will get
    home team
        last 2 games played at home ie. h_h_feature-1, h_h_feature-2
        last 2 games played away ie h_a_feature-1, h_a_feature-2
    away team
        last 2 games played at home ie a_h_feature-1, a_h_feature-2
        last 2 games played away ie a_a_feature-1, a_a_feature-2
    where -1 is the most recent game prior to the current game, and -2 is the game before that
    the current games result is in result
    """
    df = df_orig.copy(deep=True)
    keeper_cols1 = [col for col in df.columns if '-' in col]
    keeper_cols2 = [col for col in keeper_cols1 if int(col.split('-')[-1]) <= past_games]
    keeper_cols3 = [col for col in keeper_cols2 if int(col.split('-')[-1]) > 0]

    keeper_cols4 = keeper_cols3 + ['result', 'h_ftGoals', 'a_ftGoals']
    

    df = df[keeper_cols4]
    return keeper_cols3, df

PAST_GAMES = 3

keeper_cols3, df_cut = get_n_past_games(df, PAST_GAMES)

df_cut.head()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3,result,h_ftGoals,a_ftGoals
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,1.0,2.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,1.0,3.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,draw,2.0,2.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,hwin,2.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,awin,1.0,2.0


In [6]:
df_cut.tail()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3,result,h_ftGoals,a_ftGoals
4175,0.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,0.0,1.0,0.0,17.0,8.0,18.0,11.0,16.0,17.0,22.0,12.0,23.0,14.0,9.0,9.0,2.0,4.0,3.0,3.0,5.0,5.0,5.0,4.0,6.0,3.0,1.0,2.0,1.86,3.24,1.75,1.50,1.90,1.86,1.18,2.23,1.27,15.42,3.14,3.32,3.47,3.54,3.52,4.23,3.44,3.61,7.37,3.51,5.83,7.57,3.29,3.25,hwin,3.0,0.0
4176,2.0,2.0,0.0,4.0,3.0,3.0,3.0,0.0,5.0,1.0,1.0,0.0,14.0,10.0,18.0,19.0,17.0,18.0,19.0,15.0,19.0,13.0,11.0,6.0,6.0,7.0,6.0,7.0,6.0,4.0,7.0,2.0,12.0,8.0,5.0,2.0,1.71,4.37,1.74,1.32,2.63,1.35,1.13,1.09,1.13,2.50,2.58,3.08,3.90,3.61,3.67,5.44,3.52,5.12,9.25,11.57,8.89,3.20,3.24,3.40,awin,0.0,1.0
4177,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,11.0,10.0,17.0,5.0,11.0,8.0,7.0,8.0,10.0,12.0,4.0,7.0,3.0,3.0,7.0,1.0,7.0,2.0,2.0,3.0,4.0,3.0,1.0,1.0,3.04,6.97,2.53,9.73,3.91,9.57,2.65,2.28,9.98,3.54,21.51,3.29,3.20,4.23,3.15,5.54,3.41,5.64,3.31,3.20,5.28,3.50,8.89,3.15,awin,1.0,2.0
4178,0.0,3.0,5.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,0.0,20.0,20.0,13.0,6.0,9.0,12.0,11.0,8.0,13.0,10.0,9.0,10.0,2.0,7.0,10.0,3.0,1.0,3.0,4.0,5.0,4.0,5.0,3.0,3.0,1.31,1.24,1.19,21.80,3.60,4.54,4.54,6.27,1.86,3.23,1.51,1.92,5.54,6.52,7.39,9.25,3.16,3.42,3.53,4.24,3.40,3.51,4.30,3.65,hwin,4.0,0.0
4179,0.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,9.0,4.0,18.0,7.0,12.0,17.0,9.0,9.0,6.0,13.0,11.0,5.0,2.0,1.0,6.0,5.0,3.0,9.0,3.0,1.0,1.0,3.0,4.0,2.0,4.46,9.79,2.00,3.09,3.04,3.07,2.88,2.29,4.06,3.30,5.87,11.34,3.74,5.44,3.41,3.02,3.15,3.23,3.24,3.25,3.65,3.39,4.49,5.83,hwin,3.0,1.0


In [7]:
df_cut2 = df_cut.dropna(subset=keeper_cols3).reset_index(drop=True)
df_cut2.drop(columns=['h_ftGoals', 'a_ftGoals'], inplace=True)
df_cut2.head()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3,result
0,2.0,2.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,0.0,13.0,11.0,10.0,4.0,9.0,7.0,14.0,7.0,9.0,16.0,9.0,5.0,7.0,7.0,6.0,1.0,6.0,3.0,8.0,2.0,4.0,12.0,6.0,1.0,2.00,2.56,2.41,13.67,2.93,2.86,2.27,5.41,3.18,3.70,3.38,2.91,3.26,3.16,3.18,5.69,3.22,3.21,3.21,3.52,3.21,3.25,3.25,3.20,draw
1,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0,9.0,10.0,15.0,7.0,12.0,14.0,14.0,9.0,14.0,10.0,5.0,3.0,3.0,5.0,11.0,6.0,3.0,4.0,4.0,4.0,5.0,4.0,2.0,2.0,2.18,2.46,6.17,4.85,3.89,5.25,2.10,2.33,2.33,2.99,3.06,12.24,3.24,3.19,3.61,3.40,3.25,3.45,3.22,3.22,3.20,3.21,3.21,5.26,hwin
2,3.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,13.0,13.0,8.0,7.0,8.0,14.0,12.0,6.0,9.0,10.0,10.0,12.0,7.0,6.0,6.0,3.0,4.0,5.0,8.0,2.0,6.0,5.0,5.0,9.0,3.10,2.08,2.24,3.21,5.55,3.52,2.23,4.12,1.62,3.33,4.51,9.40,3.23,3.25,3.22,3.22,3.61,3.20,3.20,3.25,3.50,3.22,3.33,4.49,draw
3,1.0,3.0,1.0,3.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,1.0,13.0,18.0,13.0,13.0,10.0,6.0,14.0,25.0,14.0,8.0,8.0,14.0,7.0,12.0,9.0,8.0,3.0,3.0,5.0,14.0,8.0,3.0,3.0,7.0,2.39,2.24,2.41,2.23,6.07,2.18,2.78,1.45,1.82,2.82,4.09,3.04,3.20,3.21,3.18,3.23,3.77,3.21,3.17,3.91,3.30,3.20,3.28,3.22,draw
4,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,17.0,14.0,22.0,20.0,6.0,15.0,16.0,18.0,19.0,16.0,15.0,20.0,9.0,5.0,9.0,8.0,2.0,7.0,8.0,7.0,11.0,8.0,6.0,12.0,1.20,1.51,1.23,1.66,2.88,1.55,1.37,1.28,1.19,1.80,1.88,1.61,5.69,3.77,5.26,3.43,3.06,3.61,4.23,4.74,5.74,3.34,3.25,3.51,hwin


In [8]:
df_cut2.tail()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3,result
3492,0.0,2.0,1.0,1.0,2.0,3.0,1.0,1.0,1.0,0.0,1.0,0.0,17.0,8.0,18.0,11.0,16.0,17.0,22.0,12.0,23.0,14.0,9.0,9.0,2.0,4.0,3.0,3.0,5.0,5.0,5.0,4.0,6.0,3.0,1.0,2.0,1.86,3.24,1.75,1.50,1.90,1.86,1.18,2.23,1.27,15.42,3.14,3.32,3.47,3.54,3.52,4.23,3.44,3.61,7.37,3.51,5.83,7.57,3.29,3.25,hwin
3493,2.0,2.0,0.0,4.0,3.0,3.0,3.0,0.0,5.0,1.0,1.0,0.0,14.0,10.0,18.0,19.0,17.0,18.0,19.0,15.0,19.0,13.0,11.0,6.0,6.0,7.0,6.0,7.0,6.0,4.0,7.0,2.0,12.0,8.0,5.0,2.0,1.71,4.37,1.74,1.32,2.63,1.35,1.13,1.09,1.13,2.50,2.58,3.08,3.90,3.61,3.67,5.44,3.52,5.12,9.25,11.57,8.89,3.20,3.24,3.40,awin
3494,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,11.0,10.0,17.0,5.0,11.0,8.0,7.0,8.0,10.0,12.0,4.0,7.0,3.0,3.0,7.0,1.0,7.0,2.0,2.0,3.0,4.0,3.0,1.0,1.0,3.04,6.97,2.53,9.73,3.91,9.57,2.65,2.28,9.98,3.54,21.51,3.29,3.20,4.23,3.15,5.54,3.41,5.64,3.31,3.20,5.28,3.50,8.89,3.15,awin
3495,0.0,3.0,5.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,0.0,20.0,20.0,13.0,6.0,9.0,12.0,11.0,8.0,13.0,10.0,9.0,10.0,2.0,7.0,10.0,3.0,1.0,3.0,4.0,5.0,4.0,5.0,3.0,3.0,1.31,1.24,1.19,21.80,3.60,4.54,4.54,6.27,1.86,3.23,1.51,1.92,5.54,6.52,7.39,9.25,3.16,3.42,3.53,4.24,3.40,3.51,4.30,3.65,hwin
3496,0.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,9.0,4.0,18.0,7.0,12.0,17.0,9.0,9.0,6.0,13.0,11.0,5.0,2.0,1.0,6.0,5.0,3.0,9.0,3.0,1.0,1.0,3.0,4.0,2.0,4.46,9.79,2.00,3.09,3.04,3.07,2.88,2.29,4.06,3.30,5.87,11.34,3.74,5.44,3.41,3.02,3.15,3.23,3.24,3.25,3.65,3.39,4.49,5.83,hwin


In [9]:
#df_temp = df_cut2.iloc[:, -5:]
df_temp = df_cut2

In [10]:
df_temp.head()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3,result
0,2.0,2.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,0.0,13.0,11.0,10.0,4.0,9.0,7.0,14.0,7.0,9.0,16.0,9.0,5.0,7.0,7.0,6.0,1.0,6.0,3.0,8.0,2.0,4.0,12.0,6.0,1.0,2.00,2.56,2.41,13.67,2.93,2.86,2.27,5.41,3.18,3.70,3.38,2.91,3.26,3.16,3.18,5.69,3.22,3.21,3.21,3.52,3.21,3.25,3.25,3.20,draw
1,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0,9.0,10.0,15.0,7.0,12.0,14.0,14.0,9.0,14.0,10.0,5.0,3.0,3.0,5.0,11.0,6.0,3.0,4.0,4.0,4.0,5.0,4.0,2.0,2.0,2.18,2.46,6.17,4.85,3.89,5.25,2.10,2.33,2.33,2.99,3.06,12.24,3.24,3.19,3.61,3.40,3.25,3.45,3.22,3.22,3.20,3.21,3.21,5.26,hwin
2,3.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,13.0,13.0,8.0,7.0,8.0,14.0,12.0,6.0,9.0,10.0,10.0,12.0,7.0,6.0,6.0,3.0,4.0,5.0,8.0,2.0,6.0,5.0,5.0,9.0,3.10,2.08,2.24,3.21,5.55,3.52,2.23,4.12,1.62,3.33,4.51,9.40,3.23,3.25,3.22,3.22,3.61,3.20,3.20,3.25,3.50,3.22,3.33,4.49,draw
3,1.0,3.0,1.0,3.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,1.0,13.0,18.0,13.0,13.0,10.0,6.0,14.0,25.0,14.0,8.0,8.0,14.0,7.0,12.0,9.0,8.0,3.0,3.0,5.0,14.0,8.0,3.0,3.0,7.0,2.39,2.24,2.41,2.23,6.07,2.18,2.78,1.45,1.82,2.82,4.09,3.04,3.20,3.21,3.18,3.23,3.77,3.21,3.17,3.91,3.30,3.20,3.28,3.22,draw
4,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,17.0,14.0,22.0,20.0,6.0,15.0,16.0,18.0,19.0,16.0,15.0,20.0,9.0,5.0,9.0,8.0,2.0,7.0,8.0,7.0,11.0,8.0,6.0,12.0,1.20,1.51,1.23,1.66,2.88,1.55,1.37,1.28,1.19,1.80,1.88,1.61,5.69,3.77,5.26,3.43,3.06,3.61,4.23,4.74,5.74,3.34,3.25,3.51,hwin


In [11]:
df3 = df_temp.copy(deep=True)
df3['result'] = pd.Categorical(df3['result'])
df3['result'] = df3['result'].cat.codes
print(df3['result'].head())

0    1
1    2
2    1
3    1
4    2
Name: result, dtype: int8


In [12]:
# # Transform into a categorical variable

# darts.competitor = pd.Categorical(darts.competitor)

# # Assign a number to each category (label encoding)
# darts.competitor = darts.competitor.cat.codes 

# # Print the label encoded competitors
# print('Label encoded competitors: \n',darts.competitor.head())


# Import to_categorical from keras utils module
from keras.utils import to_categorical

# Use to_categorical on your labels
results = df3.drop(columns=['result'], axis=1)
results = to_categorical(df3['result'])
df3.drop(columns=['result'], inplace=True)

# Now print the to_categorical() result
print('One-hot encoded competitors: \n',results)

One-hot encoded competitors: 
 [[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 ...
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [13]:
df3.head()

,h_h_ftGoals-1,h_h_ftGoals-2,h_h_ftGoals-3,a_a_ftGoals-1,a_a_ftGoals-2,a_a_ftGoals-3,a_h_ftGoals-1,a_h_ftGoals-2,a_h_ftGoals-3,h_a_ftGoals-1,h_a_ftGoals-2,h_a_ftGoals-3,h_h_shots-1,h_h_shots-2,h_h_shots-3,a_a_shots-1,a_a_shots-2,a_a_shots-3,a_h_shots-1,a_h_shots-2,a_h_shots-3,h_a_shots-1,h_a_shots-2,h_a_shots-3,h_h_shotsOnTarget-1,h_h_shotsOnTarget-2,h_h_shotsOnTarget-3,a_a_shotsOnTarget-1,a_a_shotsOnTarget-2,a_a_shotsOnTarget-3,a_h_shotsOnTarget-1,a_h_shotsOnTarget-2,a_h_shotsOnTarget-3,h_a_shotsOnTarget-1,h_a_shotsOnTarget-2,h_a_shotsOnTarget-3,h_h_oddsWinMean-1,h_h_oddsWinMean-2,h_h_oddsWinMean-3,a_a_oddsWinMean-1,a_a_oddsWinMean-2,a_a_oddsWinMean-3,a_h_oddsWinMean-1,a_h_oddsWinMean-2,a_h_oddsWinMean-3,h_a_oddsWinMean-1,h_a_oddsWinMean-2,h_a_oddsWinMean-3,h_h_oddsDrawMean-1,h_h_oddsDrawMean-2,h_h_oddsDrawMean-3,a_a_oddsDrawMean-1,a_a_oddsDrawMean-2,a_a_oddsDrawMean-3,a_h_oddsDrawMean-1,a_h_oddsDrawMean-2,a_h_oddsDrawMean-3,h_a_oddsDrawMean-1,h_a_oddsDrawMean-2,h_a_oddsDrawMean-3
0,2.0,2.0,1.0,0.0,0.0,2.0,2.0,0.0,1.0,0.0,2.0,0.0,13.0,11.0,10.0,4.0,9.0,7.0,14.0,7.0,9.0,16.0,9.0,5.0,7.0,7.0,6.0,1.0,6.0,3.0,8.0,2.0,4.0,12.0,6.0,1.0,2.00,2.56,2.41,13.67,2.93,2.86,2.27,5.41,3.18,3.70,3.38,2.91,3.26,3.16,3.18,5.69,3.22,3.21,3.21,3.52,3.21,3.25,3.25,3.20
1,0.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0,1.0,1.0,0.0,0.0,9.0,10.0,15.0,7.0,12.0,14.0,14.0,9.0,14.0,10.0,5.0,3.0,3.0,5.0,11.0,6.0,3.0,4.0,4.0,4.0,5.0,4.0,2.0,2.0,2.18,2.46,6.17,4.85,3.89,5.25,2.10,2.33,2.33,2.99,3.06,12.24,3.24,3.19,3.61,3.40,3.25,3.45,3.22,3.22,3.20,3.21,3.21,5.26
2,3.0,1.0,2.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,13.0,13.0,8.0,7.0,8.0,14.0,12.0,6.0,9.0,10.0,10.0,12.0,7.0,6.0,6.0,3.0,4.0,5.0,8.0,2.0,6.0,5.0,5.0,9.0,3.10,2.08,2.24,3.21,5.55,3.52,2.23,4.12,1.62,3.33,4.51,9.40,3.23,3.25,3.22,3.22,3.61,3.20,3.20,3.25,3.50,3.22,3.33,4.49
3,1.0,3.0,1.0,3.0,0.0,0.0,1.0,4.0,1.0,0.0,1.0,1.0,13.0,18.0,13.0,13.0,10.0,6.0,14.0,25.0,14.0,8.0,8.0,14.0,7.0,12.0,9.0,8.0,3.0,3.0,5.0,14.0,8.0,3.0,3.0,7.0,2.39,2.24,2.41,2.23,6.07,2.18,2.78,1.45,1.82,2.82,4.09,3.04,3.20,3.21,3.18,3.23,3.77,3.21,3.17,3.91,3.30,3.20,3.28,3.22
4,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,3.0,1.0,0.0,1.0,17.0,14.0,22.0,20.0,6.0,15.0,16.0,18.0,19.0,16.0,15.0,20.0,9.0,5.0,9.0,8.0,2.0,7.0,8.0,7.0,11.0,8.0,6.0,12.0,1.20,1.51,1.23,1.66,2.88,1.55,1.37,1.28,1.19,1.80,1.88,1.61,5.69,3.77,5.26,3.43,3.06,3.61,4.23,4.74,5.74,3.34,3.25,3.51


In [14]:
X_train, X_test, y_train, y_test = train_test_split(df3, results, test_size=0.2)

In [15]:
X_train.shape

(2797, 60)

In [16]:
y_train.shape

(2797, 3)

In [17]:
X_test.shape

(700, 60)

In [18]:
# Instantiate a sequential model
model = Sequential()
  
# Add 3 dense layers of 128, 64 and 32 neurons each
shape1 = X_test.shape[1]
model.add(Dense(128, input_shape=(shape1,), activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
  
# Add a dense layer with as many neurons as competitors
model.add(Dense(3, activation='softmax'))
  
# Compile your model using categorical_crossentropy loss
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
# Train your model on the training data for 200 epochs
history = model.fit(X_train,y_train, epochs=30)


Epoch 1/30
2797/2797 [==============================] - 0s 166us/step - loss: 1.1085 - accuracy: 0.4619
Epoch 2/30
2797/2797 [==============================] - 0s 52us/step - loss: 1.0140 - accuracy: 0.5088
Epoch 3/30
2797/2797 [==============================] - 0s 51us/step - loss: 1.0068 - accuracy: 0.5098
Epoch 4/30
2797/2797 [==============================] - 0s 54us/step - loss: 0.9875 - accuracy: 0.5209
Epoch 5/30
2797/2797 [==============================] - 0s 55us/step - loss: 0.9720 - accuracy: 0.5345
Epoch 6/30
2797/2797 [==============================] - 0s 53us/step - loss: 0.9696 - accuracy: 0.5324
Epoch 7/30
2797/2797 [==============================] - 0s 52us/step - loss: 0.9671 - accuracy: 0.5434
Epoch 8/30
2797/2797 [==============================] - 0s 52us/step - loss: 0.9596 - accuracy: 0.5417
Epoch 9/30
2797/2797 [==============================] - 0s 52us/step - loss: 0.9413 - accuracy: 0.5556
Epoch 10/30
2797/2797 [==============================] - 0s 53us/step -

In [20]:
# Evaluate your model accuracy on the test data
accuracy = model.evaluate(X_test, y_test)[1]

# Print accuracy
print('Accuracy:', accuracy)

700/700 [==============================] - 0s 53us/step
Accuracy: 0.4571428596973419


In [21]:
preds = model.predict_proba(X_test)

In [22]:
preds

array([[0.01062204, 0.00784292, 0.9815351 ],
       [0.46344003, 0.07237277, 0.46418715],
       [0.00080271, 0.214309  , 0.78488827],
       ...,
       [0.01090405, 0.4325741 , 0.5565219 ],
       [0.6987785 , 0.24534217, 0.05587936],
       [0.06790157, 0.8855038 , 0.04659467]], dtype=float32)

In [23]:
y_test

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)